In [1]:
import pandas as pd
import numpy as np
import itertools

# Visualización de grafos
from pyvis.network import Network

# Interacciones en el notebook
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# Cargo los datos de las interacciones
artworks_data = pd.read_csv('/data/Prado_artworks_wikidata.csv')
interactions_df = pd.read_csv('/data/Prado_emotions.csv')

# Creo diccionarios para obtener imagenes y títulos de los cuadros
artwork_ids = np.unique(interactions_df.artworkId.values)
artworks_dict = dict()
images_dict = dict()
for art_id in artwork_ids:
    title = artworks_data[artworks_data['ID'] == art_id]['Title'].values[0]
    image = artworks_data[artworks_data['ID'] == art_id]['Image URL'].values[0]
    artworks_dict[art_id] = title
    images_dict[art_id] = image
    
# Create array of emotions and its colors
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']
emotions = np.unique(interactions_df['emotion'].values)
emotions_colors = dict()
for i in range(len(emotions)):
    emotions_colors[emotions[i]] = colors[i]

In [3]:
def get_users_relations(df, emotion=None):
    """
    Función para obtener las relaciones que tienen los usuarios entre sí. Es una lista que, en cada elemento,
    se muestra una lista de los usuarios que han interactuado con la misma emoción (o emoción seleccionada)
    con un mismo cuadro.
    """
    if emotion == None:
        return df.groupby(by=['artworkId', 'emotion'])['userId'].apply(list).values
    else:
        aux_df = df[df['emotion'] == emotion]
        return aux_df.groupby(by=['artworkId', 'emotion'])['userId'].apply(list).values

In [4]:
def get_users_edges(df, emotion=None):
    users_relations = get_users_relations(df, emotion)
    
    # Obtengo todos los pares de relaciones
    relations_pairs = [[pair for pair in itertools.combinations(relations, 2)] for relations in users_relations]
    edges = [edge for relations in relations_pairs for edge in relations]
    
    # Obtengo el peso de cada eje
    edges_dict = dict()
    for edge in edges:
        if edge in edges_dict:
            edges_dict[edge] += 1
        elif (edge[1], edge[0]) in edges_dict:
            edges_dict[(edge[1], edge[0])] += 1
        else:
            edges_dict[edge] = 1
    
    # Creo la lista final de ejes (from, to, weigth)
    user_edges = []
    for i in edges_dict:
        user_edges.append((i[0], i[1], edges_dict[i]))
        
    return user_edges

In [5]:
def get_artworks_relations(df, emotion=None):
    """
    Función para obtener las relaciones que tienen los cuadros entre sí. Es una lista que, en cada elemento,
    se muestra una lista de los cuadros que han recibido la misma emoción (o emoción seleccionada)
    por parte del mismo usuario.
    """
    if emotion == None:
        return df.groupby(by=['userId', 'emotion'])['artworkId'].apply(list).values
    else:
        aux_df = df[df['emotion'] == emotion]
        return aux_df.groupby(by=['userId', 'emotion'])['artworkId'].apply(list).values

In [6]:
def get_artworks_edges(df, emotion=None):
    artworks_relations = get_artworks_relations(df, emotion)
    
    # Obtengo todos los pares de relaciones
    relations_pairs = [[pair for pair in itertools.combinations(relations, 2)] for relations in artworks_relations]
    edges = [edge for relations in relations_pairs for edge in relations]
    
    # Obtengo el peso de cada eje
    edges_dict = dict()
    for edge in edges:
        if edge in edges_dict:
            edges_dict[edge] += 1
        elif (edge[1], edge[0]) in edges_dict:
            edges_dict[(edge[1], edge[0])] += 1
        else:
            edges_dict[edge] = 1
    
    # Creo la lista final de ejes (from, to, weigth)
    artwork_edges = []
    for i in edges_dict:
        artwork_edges.append((i[0], i[1], edges_dict[i]))
        
    return artwork_edges

In [7]:
def show_graph_users(emotion=None, show_physics=False):
    nodes = np.unique(interactions_df['userId'].values)
    edges = get_users_edges(interactions_df, emotion)

    vis = Network(notebook=True, width="1000px", height="800px")

    for n in nodes:
        vis.add_node(str(n), size=1)

    for e in edges:
        vis.add_edge(str(e[0]), str(e[1]), value=e[2])
    
    if show_physics:
        vis.show_buttons(filter_=['physics'])
        
    return vis.show('users_graph.html')

In [8]:
def show_graph_artworks(emotion=None, show_physics=False):
    nodes = np.unique(interactions_df['artworkId'].values)
    edges = get_artworks_edges(interactions_df, emotion)
    
    vis = Network(notebook=True, width="1000px", height="800px")

    for n in nodes:
        vis.add_node(str(artworks_dict[n]), size=30, shape='image', image =images_dict[n])

    for e in edges:
        vis.add_edge(str(artworks_dict[e[0]]), str(artworks_dict[e[1]]), value=e[2])

    if show_physics:
        vis.show_buttons(filter_=['physics'])
        
    return vis.show('artworks_graph.html')

In [9]:
emotions_options = list(emotions)
emotions_options.append(None)

In [10]:
@interact_manual
def show_graps(nodes=['artworks', 'users'], emotion=emotions_options, show_physics=False):
    
    if nodes == 'artworks':
        return show_graph_artworks(emotion, show_physics)
    else:
        return show_graph_users(emotion, show_physics)

interactive(children=(Dropdown(description='nodes', options=('artworks', 'users'), value='artworks'), Dropdown…